In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import time
%matplotlib inline  
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/jz2308/bus-Capstone')

# these two modules are homemade
import gtfs
import arrivals
import ttools
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('2015-12-03','gtfs/')
stops = gtfs.load_stops('2015-12-03','gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('2015-12-03','gtfs/')
tcal=gtfs.TransitCalendar('2015-12-03')
thurs_services = tcal.get_service_ids('2015-12-03')
thurs_trips = trips.service_id.isin(thurs_services)
thurs_stops = stop_times.reset_index().set_index('trip_id').loc[thurs_trips]
print 'Finished loading GTFS data.'

Finished loading GTFS data.


In [3]:
interpolated = pd.read_csv('2015-12-03_interpolated.csv')

In [4]:
interpolated.head()

,route,trip_id,trip_date,vehicle_id,stop_id,interpolated_arrival_time
0,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,402805,0 days 06:10:56.778916
1,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,405057,0 days 06:14:24.977703
2,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,403416,0 days 06:16:58.526210
3,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,402809,0 days 06:17:25.226045
4,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,402811,0 days 06:18:36.819142


In [5]:
interpolated['interpolated_arrival_time'] = pd.to_timedelta(interpolated['interpolated_arrival_time'])

In [6]:
interpolated = interpolated.merge(stop_times[['arrival_time','stop_sequence']],how='left',left_on=['trip_id','stop_id'],right_index=True)

In [7]:
# bools = interpolated.duplicated(subset=['route','trip_id','trip_date','vehicle_id','stop_id'],take_last=False) + interpolated.duplicated(subset=['route','trip_id','trip_date','vehicle_id','stop_id'],take_last=True)
# interpolated[bools].sort(['route','trip_id','trip_date','vehicle_id']).shape

In [8]:
interpolated.set_index(['route','trip_id','trip_date','vehicle_id','stop_sequence','stop_id'],inplace=True)

## Get the densest stop_id by shape (trip?)

In [9]:
trip_groups = interpolated.groupby(level=(0,2,5,4)).size()

In [10]:
def series_max(s):
    return s.idxmax()[2]

In [11]:
densest_stops = trip_groups.groupby(level=(0)).apply(series_max)
densest_stops

route
MTA NYCT_B1      300009
MTA NYCT_B11     301173
MTA NYCT_B12     301328
MTA NYCT_B13     301505
MTA NYCT_B14     301598
MTA NYCT_B15     301135
MTA NYCT_B16     301677
MTA NYCT_B17     300853
MTA NYCT_B2      300173
MTA NYCT_B20     301959
MTA NYCT_B24     503993
MTA NYCT_B25     302383
MTA NYCT_B26     302372
MTA NYCT_B3      300261
MTA NYCT_B31     300175
MTA NYCT_B32     302304
MTA NYCT_B35     302692
MTA NYCT_B36     300019
MTA NYCT_B37     302914
MTA NYCT_B38     302948
MTA NYCT_B39     403679
MTA NYCT_B4      300368
MTA NYCT_B41     303295
MTA NYCT_B42     303358
MTA NYCT_B43     303711
MTA NYCT_B44     303483
MTA NYCT_B44+    303441
MTA NYCT_B45     303531
MTA NYCT_B46     303592
MTA NYCT_B47     300857
                  ...  
MTABC_Q50        101743
MTABC_Q52        550070
MTABC_Q53        550070
MTABC_Q60        504399
MTABC_Q64        551812
MTABC_Q65        550032
MTABC_Q66        551894
MTABC_Q67        503832
MTABC_Q69        550010
MTABC_Q70        504542
MTABC_Q72 

## Function that can applied repeatedly

In [12]:
# convert scheduled time to numeric
interpolated['schedule_convert'] = pd.to_timedelta(interpolated['arrival_time'])

In [13]:
def peak_hour(x):
    if x > '06:00:00' and x < '09:00:00':
        return True
    else:
        if x > '16:00:00' and x < '19:00:00':
            return True
        else:
            return False

In [14]:
def wait_ass(x,y):
    if x == True:
        if y >= 3*60:
            return True
        else:
            return False
    if x == False:
        if y>= 5*60:
            return True
        else:
            return False         

In [25]:
def data_pre(route,stop):
    pd.options.mode.chained_assignment = None
    interpolated_2 = interpolated.xs((route,stop),level=(0,5),drop_level=False)
    interpolated_2['sche_headway'] = interpolated_2.sort(columns='schedule_convert')['schedule_convert'].diff()
    interpolated_2['inter_headway'] = interpolated_2.sort(columns='interpolated_arrival_time')['interpolated_arrival_time'].diff()
    interpolated_2['sche_h_sec'] = interpolated_2['sche_headway']/ttools.datetime.timedelta(seconds=1)
    interpolated_2['inter_h_sec'] = interpolated_2['inter_headway']/ttools.datetime.timedelta(seconds=1)
    interpolated_2['P_hour'] = map(lambda x:peak_hour(x),interpolated_2['arrival_time'])
    interpolated_2['diff'] = interpolated_2['inter_h_sec'] - interpolated_2['sche_h_sec']
    interpolated_2['wait_ass'] = map(lambda x,y:wait_ass(x,y), interpolated_2['P_hour'],interpolated_2['diff'])
    ##wait assessment for the single route on the single stop
    # 100% is best
    #return 1.0-(1.0*interpolated_2['wait_ass'].sum()/len(interpolated_2['wait_ass']))
    return interpolated_2

In [26]:
trydata = data_pre('MTA NYCT_B1',300009)

In [27]:
###Only report for subway in MTA annual repo, permissible when the real arrival time within +-50% of the schedule headway
def headway_regularity(sche,inter):
    if inter >= 0.5*sche and inter <= 1.5*sche:
        return True
    else:
        return False    

In [28]:
trydata['headway_reg']=map(lambda x,y:headway_regularity(x,y), trydata['sche_h_sec'],trydata['inter_h_sec'])

In [30]:
## on-time performance
## 1 min before or 5 min later compared with the schedule
## According to http://comptroller.nyc.gov/wp-content/uploads/documents/MH13_118A.pdf
## http://web.mta.info/developers/performance.html

In [48]:
def On_Time_Per(sche,inter):
    if inter > sche - 60 and inter <= sche + 300:
        return True
    else:
        return False

In [49]:
trydata['OnTimeP'] = map(lambda x,y:On_Time_Per(x,y), trydata['sche_h_sec'],trydata['inter_h_sec'] )

In [50]:
trydata

interpolated_arrival_time  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                              
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009             12:21:25.050014   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009             09:50:21.445500   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009             17:51:37.570228   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009             21:32:51.901086   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009             15:12:25.500396   

                                                                                           arrival_time  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009      12:23:02   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009      09:43:02   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009      17:55:02   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009      21:33:19   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009      15:12:02   

                                                                                            schedule_convert  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                     
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009           12:23:02   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009           09:43:02   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009           17:55:02   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009           21:33:19   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009           15:12:02   

                                                                                            sche_headway  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                 
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009       00:10:00   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009       00:10:00   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009       00:05:00   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009       00:05:00   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009       00:02:00   

                                                                                             inter_headway  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                   
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009  00:10:01.374255   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009  00:19:02.201000   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009  00:02:24.854065   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009  00:05:55.430150   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009  00:00:55.234394   

                                                                                            sche_h_sec  \
route       trip_id                         trip_date  vehicle_id    stop_s

In [ ]:
## Excess Wait Time
## According to http://greatergreaterwashington.org/post/2616/bus-on-time-performance-it-matters-what-you-measure/

In [51]:
OTP = (1.0*trydata['OnTimeP'].sum())/len(trydata)
print OTP

0.570552147239


In [61]:
Headway_Regularity = (1.0*trydata['headway_reg'].sum())/len(trydata)
print Headway_Regularity

0.680981595092


In [80]:
## Trip Length for each trip, we need to compare the schedule trip length with actual trip length
## I saw the trip length distribution in Sponsor Report ppt, so I didn't add it here

In [81]:
## Headway Adherence
import numpy as np
def headway_adherence(sche,inter):
    return 1.0* np.absolute(sche-inter)/sche

In [82]:
trydata['headway_adh'] = map(lambda x,y: headway_adherence(x,y), trydata['sche_h_sec'],trydata['inter_h_sec'])

In [83]:
trydata.head()

interpolated_arrival_time  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                              
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009             12:21:25.050014   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009             09:50:21.445500   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009             17:51:37.570228   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009             21:32:51.901086   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009             15:12:25.500396   

                                                                                           arrival_time  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009      12:23:02   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009      09:43:02   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009      17:55:02   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009      21:33:19   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009      15:12:02   

                                                                                            schedule_convert  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                     
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009           12:23:02   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009           09:43:02   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009           17:55:02   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009           21:33:19   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009           15:12:02   

                                                                                            sche_headway  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                 
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009       00:10:00   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009       00:10:00   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009       00:05:00   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009       00:05:00   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009       00:02:00   

                                                                                             inter_headway  \
route       trip_id                         trip_date  vehicle_id    stop_sequence stop_id                   
MTA NYCT_B1 UP_D5-Weekday-SDon-073300_B1_12 2015-12-03 MTA NYCT_4892 8             300009  00:10:01.374255   
            UP_D5-Weekday-SDon-057800_B1_8  2015-12-03 MTA NYCT_7168 8             300009  00:19:02.201000   
            UP_D5-Weekday-SDon-106500_B1_43 2015-12-03 MTA NYCT_7172 8             300009  00:02:24.854065   
            UP_D5-Weekday-SDon-128900_B1_42 2015-12-03 MTA NYCT_4867 8             300009  00:05:55.430150   
            UP_D5-Weekday-SDon-090200_B1_35 2015-12-03 MTA NYCT_5118 8             300009  00:00:55.234394   

                                                                                            sche_h_sec  \
route       trip_id                         trip_date  vehicle_id    stop_s

In [ ]:
## If the adherence function was applied on the beginning and ending time of the whole trip, we can get the running time adherence for a trip